In [ ]:
!pip install langchain langchain-community
!pip install langchain-google-genai
!pip install langchain-qdrant

## Setup the API key

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## External Data

In [ ]:
# Retrieval
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# Vector Database
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
# Augmentation
from langchain_core.prompts import PromptTemplate
# Generation
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser

In [ ]:
URLs = [
    "https://www.amazon.in/gp/help/customer/display.html?nodeId=GDF5PQP4Z6SUH4CQ",
    "https://www.amazon.in/gp/help/customer/display.html?ref_=hp_left_v4_sib&nodeId=GW2QNFK3F7T8ULDS"
]

## Load the data and chunk it

In [ ]:
loader = WebBaseLoader(
    web_paths = URLs,
)
docs = loader.load()

In [ ]:
for data_content in docs:
    data_content.page_content = data_content.page_content.replace("\n", " ")

In [ ]:
docs

[Document(metadata={'source': 'https://www.amazon.in/gp/help/customer/display.html?nodeId=GDF5PQP4Z6SUH4CQ', 'title': 'Ordering with Amazon - FAQ - Amazon Customer Service', 'description': 'Know about how to place an order with Amazon, how to cancel or modify your orders and how to change the payment methods To check the below information in your preferred regional language, select the link: English, हिन्दी (Hindi), தமிழ் (Tamil), తెలుగు (Telugu), ಕನ್ನಡ (Kannada), മലയാളം (Malayalam), मराठी (Marathi), বাংলা (Bengali)', 'language': 'en-in'}, page_content='                                Ordering with Amazon - FAQ - Amazon Customer Service                                                Shortcuts menu   Skip to            Main content                  Keyboard shortcuts        Search  alt + /        Cart  shift + alt + C        Home  shift + alt + H        Orders  shift + alt + O        Open/close shortcuts menu  shift + alt + Z           To move between items, use your keyboard\'s up or d

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 0,
)

In [ ]:
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

26

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [ ]:
try:
  client = QdrantClient(path="/db")

  client.create_collection(
    collection_name="rag",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
  )
except RuntimeError as e:
  print("Qdrant client already exists")

In [ ]:
!ls /db/collection/

rag


In [ ]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="rag",
    embedding=embeddings,
)

In [ ]:
vector_store.add_documents(chunks)

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k":4})

In [ ]:
user_query = "What happens if I want to return the item purchased using Coupon?"
retriever.invoke(user_query)

[Document(metadata={'source': 'https://www.amazon.in/gp/help/customer/display.html?ref_=hp_left_v4_sib&nodeId=GW2QNFK3F7T8ULDS', 'title': 'Coupons - FAQ - Amazon Customer Service', 'description': 'Amazon coupons help you save big on your purchases. Collect from a variety of coupons available on our website and get instant discount at the time of checkout. [[[HELP_REGIONAL_995846-T1]]]To check the below information in your preferred regional language, select the link: English, हिन्दी (Hindi), தமிழ் (Tamil), తెలుగు (Telugu), ಕನ್ನಡ (Kannada), മലയാളം (Malayalam), मराठी (Marathi), বাংলা (Bengali).[[[HELP_REGIONAL_995846-T1]]]', 'language': 'en-in', '_id': 'b2ffa01d78384a81a07af104d4f26874', '_collection_name': 'rag'}, page_content="the discount will be automatically applied as you proceed to checkout.   4. Can I collect a coupon now and make the purchase later? Yes. The collected coupon will be saved to your account and will be automatically             applied when you purchase the item fr

In [ ]:
template = """You are an customer service assistant for question-answering tasks.
Use the following pieces of retrieved CONTEXT to answer the QUESTION.
If you don't know the answer, just say that you don't know.
---------------------
CONTEXT: {context}
---------------------
Given this information, please answer the QUESTION: {query}.
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [ ]:
llm.invoke("What is the capital of India?").content

'The capital of India is **New Delhi**.'

In [ ]:
output_parser = StrOutputParser()

chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "query": RunnablePassthrough(),
    }
    | prompt
    | llm
    | output_parser
)

In [ ]:
for chunk in chain.stream(user_query):
  print(chunk, end="", flush=True)

If you return the item within the return window and want a refund, you'll get the amount you paid. If you want a replacement, you'll get it for free without any additional payment.


In [ ]:
response = chain.invoke("who is Virat Kohli?")

In [ ]:
print(response)

I'm sorry, but the provided context does not contain information about Virat Kohli.
